# Lab 5: A Black-Box Adversarial Attack
In this lab, we'll carry out an adversarial attack in the **black box** case: we don't have the weights of the network we're trying to trick (the **target model**), and instead we can only give it inputs and see its outputs.

Black-box adversarial attacks are based on the fact that adversarial examples are _transferable_: if it tricks one network, it will likely trick another (but less well).
So, since we need network weights to make adversarial examples, we train a second model, the **surrogate model**, to act like the target model.
To do this, we'll use the target model as an "oracle" to make a dataset where the features are from real examples, and the labels are the outputs of the target model on those examples.
If the surrogate model learns to output the same probabilities as the target model on a given input, its internal representations are likely similar.
Then, when we make an adversarial example for the surrogate model, it should also trick the target model.

The end result of this lab should be an image which looks close to some image in the dataset, but tricks the target network into assigning high confidence to some other class. 

## Section 0: Load the data
This uses the same dataset as last time, so just copy the data over here in the same format.
The same data loading and preprocessing as before is in place.

In [26]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Seed numpy rng for reproducibility
np.random.seed(1337)

# Load data
x_all = np.load('X.npy')
y_all = np.load('Y.npy')

# Maps dataset-provided label to true label
label_map = {0:9, 1:0, 2:7, 3:6, 4:1, 5:8, 6:4, 7:3, 8:2, 9:5}

# Correct dataset labels
for row in range(y_all.shape[0]):
    dataset_label = np.where(y_all[row])[0][0]
    y_all[row, :] = np.zeros(10)
    y_all[row, label_map[dataset_label]] = 1
    
# Shuffle features and targets together
# Credit for this technique to:
# https://stackoverflow.com/questions/4601373/
# better-way-to-shuffle-two-numpy-arrays-in-unison
rng_state = np.random.get_state()
np.random.shuffle(x_all)
np.random.set_state(rng_state)
np.random.shuffle(y_all)

# Add a dummy channel axis to input images
x_all = np.expand_dims(x_all, axis=-1)

# Center and rescale data to the range [-1, 1]
x_all = x_all - 0.5
x_all = x_all * 2

# Create a validation set from 30% of the available data
n_points = x_all.shape[0]
n_test = int(n_points * 0.3)
n_train = n_points - n_test
x_train, x_test = np.split(x_all, [n_train], axis=0)
y_train, y_test = np.split(y_all, [n_train], axis=0)

## Section 1: Train the black-box model
Below, I've written a CNN in Keras to classify images from the dataset, and the code to train it.
This will act as the "black box model."

Train the model using the code below.
It should hit about 95-96% validation accuracy on the dataset.

In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPool2D

inputs = Input(shape=(64, 64, 1))
layer = Conv2D(16, 5, strides=(2, 2), activation='relu')(inputs)
layer = Conv2D(16, 3, activation='relu')(layer)
layer = MaxPool2D()(layer)

layer = Conv2D(32, 3, activation='relu')(layer)
layer = Conv2D(32, 3, activation='relu')(layer)
layer = MaxPool2D()(layer)

layer = Conv2D(64, 3, padding='same', activation='relu')(layer)
layer = Conv2D(64, 3, padding='same', activation='relu')(layer)
layer = MaxPool2D()(layer)

layer = Flatten()(layer)
layer = Dense(128, activation='relu')(layer)
probs = Dense(10, activation='softmax')(layer)

target_model = Model(inputs, probs)
target_model.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 16)        416       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0   

In [4]:
from keras.optimizers import Adam

opt = Adam(1e-3)
target_model.compile(opt, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
target_model.fit(x_train, y_train, 
                 validation_data=(x_test, y_test),
                 epochs=15)

Train on 1444 samples, validate on 618 samples
Epoch 1/15
1444/1444 [==============================] - 4s 3ms/step - loss: 2.0117 - accuracy: 0.2922 - val_loss: 1.2303 - val_accuracy: 0.6100
Epoch 2/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.9038 - accuracy: 0.6842 - val_loss: 0.6036 - val_accuracy: 0.7913
Epoch 3/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.4992 - accuracy: 0.8317 - val_loss: 0.3961 - val_accuracy: 0.8576
Epoch 4/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.3259 - accuracy: 0.8830 - val_loss: 0.3413 - val_accuracy: 0.8964
Epoch 5/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.1915 - accuracy: 0.9418 - val_loss: 0.3203 - val_accuracy: 0.9094
Epoch 6/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.1633 - accuracy: 0.9529 - val_loss: 0.2581 - val_accuracy: 0.9159
Epoch 7/15
1444/1444 [==============================] - 3s 2ms/step - loss: 0.0755 - accu

## Section 2: Build an "oracle dataset" and data pipeline
Now, we'll treat the target model as a black box: pretend we don't have access to its weights or its original input data.
In the wild, all we can do is give it inputs and see its outputs.

We want to train a surrogate model to act similarly to the target model, so create a fake "oracle dataset" where the features are `x_all` and the outputs are the 10-vectors of probability the target model predicts for that input.

Then, set up any `tf.data.Dataset` objects you need.
In this case we don't have a test set, just a training set. Add `tf.summary.SummaryWriter`s to save logs to `./logs/surrogate` and `./logs/adversarial`.

In [6]:
x_all_od = x_all
y_od = target_model.predict(x_all_od)

dataset_train = tf.data.Dataset.from_tensor_slices((x_all_od, y_od)).shuffle(x_all_od.shape[0]).batch(64).cache()

surr_writer = tf.summary.create_file_writer('./logs/surrogate')
adv_writer = tf.summary.create_file_writer('./logs/adversarial')

## Section 3: Build a surrogate model
The surrogate model is designed to act similarly to the target model, so that adversarial examples we create for it will also work on the target model.
Feel free to use any architecture you want, but matching the target architecture closely (maybe with a bit more capacity) is a good bet.

Copy code from last week's assignment liberally.
Most likely, you won't have to write all that much new code in this section.

This should involve tensors that:
 - Act as hidden convolutional and dense layers
 - Compute the logits and probabilities for a batch of input images
 - Compute the predicted digit from the probabilities
 - Compute the mean cross-entropy loss over a batch

### 3.1 Dense and convolutional layers
Copy the `Dense` and `Conv` classes you wrote last week here. You will need them to construct the graph of your surrogate model.

In [7]:
class Dense(tf.Module):
    '''
    Creates a dense layer module.
    
    Parameters
    ----------
    dim_input: int
        Number of features in the input representation.
    dim_output: int
        Number of features in the output representation.
        Equivalently, number of units in this layer.
    do_activation: bool
        Whether or not to apply ReLU activation.
    postfix: string
        Postfix on name scopes in this layer.
        Used to simplify visualizations.
    name: string
        Name of layer.
        
    Returns
    -------
    A tensor representing the activations of this layer.
    '''
    def __init__(self, dim_input, dim_output, do_activation=True, postfix='', name=None):
        super().__init__(name=name)
        with tf.name_scope('dense' + postfix):
            self.weights = tf.Variable(tf.initializers.he_uniform()(shape = (dim_input, dim_output), dtype = tf.float32), name = 'weights-' + postfix)
            self.bias = tf.Variable(tf.zeros_initializer()(shape = dim_output, dtype = tf.float32), name = 'bias-' + postfix)
            self.do_activation = do_activation
        
    def __call__(self, x):
        output = tf.math.add(tf.matmul(x, self.weights), self.bias)
        if self.do_activation:
            activations = tf.nn.relu(output)
            return activations
        return output
    
    
class Conv(tf.Module):
    '''
    Creates a convolutional layer module.
    
    Parameters
    ----------
    input_channels: int
        Number of channels in the input representation.
    n_filters: int
        Number of channels in the output representation.
        Equivalently, number of filters in this layer.
    filter_size: int
        Width and height of each kernel in the layer's filters.
    stride: int
        Stride to use in the x and y directions for the
        convolution operation.
    do_activation: bool
        Whether or not to apply ReLU activation.
    pool_size: int
        If > 1, does max pooling of this size to the
        width and height axes of the activation.
    postfix: string
        Postfix on name and variable scopes in this layer.
        Used to simplify visualizations.
    name: string
        Name of layer
        
    Returns
    -------
    A tensor representing the activations of this layer.
    '''
    def __init__(self, input_channels, n_filters, 
                 filter_size=3, stride=1,
                 do_activation=True, pool_size=1,
                 postfix='', name=None):
        super().__init__(name=name)
        with tf.name_scope('conv' + postfix):
            self.filter_ = tf.Variable(tf.initializers.he_uniform()(shape = (filter_size, filter_size, input_channels, n_filters), dtype = tf.float32), name = 'filter')
            self.bias = tf.Variable(tf.zeros_initializer()(shape = (n_filters), dtype = tf.float32), name = 'bias')
            self.pool_size = pool_size
            self.do_activation = do_activation
            self.stride = stride
            
    def image(self, x):
        pre_activ = tf.nn.conv2d(x, self.filter_, strides = [1, self.stride, self.stride, 1], padding = 'SAME')
        pre_activ = tf.nn.bias_add(pre_activ, self.bias)
        if self.do_activation:
            pre_activ = tf.nn.relu(pre_activ)
        return pre_activ
            
    def __call__(self, x):
        output = self.image(x)
        if self.pool_size > 1:
            output = tf.nn.max_pool(output, ksize = [1, self.pool_size, self.pool_size, 1], strides = [1, self.pool_size, self.pool_size, 1], padding = 'VALID')
        return output

### 3.2: The rest of the model
Write a `tf.Module` class called `Surrogate` that uses your classes from last week to be your surrogate model.

#### 3.2.1: Adversarial noise
This is the only part of the model that really differs from last week, and it's a bit tricky.

Add a variable to the graph that represents the adversarial noise we'll add to one example.
It should be the shape of a single input image and initialized to zeros.
In addition, pass (to the `Variable()` constructor) the keyword argument `constraint=lambda x: tf.clip_by_value(x, -0.3, 0.3)`.
Every time the variable is updated, it will become the result of the lambda function, which in this case constrains its pixel values to be in the range -0.3 to 0.3.
(Equivalently, at every step we re-project the adversarial noise back into a hypercube at the origin with side length 0.6).
I found that 0.3 works well, but feel free to change this value -- smaller values will produce less obvious noise, while larger values will produce more successful attacks. 

Add a boolean `tf.Variable` (default False) named "use_noise" to the graph.
This will act as a "switch" controlling whether a given run uses adversarial noise. Additionally, add the argument `trainable=False` to the constructor, as it makes no sense to update this variable during training.

Add a `tf.cond()` operation that, depending on `use_noise`, switches between the input image and the image plus the adversarial noise.
This allows us to train the surrogate model without adversarial noise, then enable it when crafting the adversarial example.

In [20]:
class Surrogate(tf.Module):
    def __init__(self, name = None):
        self.use_noise = tf.Variable(False, shape = (), trainable = False)
        self.noise = tf.Variable(np.zeros((64, 64)), constraint = lambda x: tf.clip_by_value(x, -0.3, 0.3), dtype = tf.float32)
        
        self.conv1 = Conv(1, 20, filter_size = 5, stride = 1, do_activation = True, pool_size = 1, postfix = 'conv1')
        self.conv2 = Conv(20, 20, filter_size = 3, stride = 1, do_activation = True, pool_size = 2, postfix = 'conv2')
        
        self.conv3 = Conv(20, 40, filter_size = 3, stride = 1, do_activation = True, pool_size = 1, postfix = 'conv3')
        self.conv4 = Conv(40, 40, filter_size = 3, stride = 1, do_activation = True, pool_size = 2, postfix = 'conv4')
        
        self.conv5 = Conv(40, 80, filter_size = 3, stride = 1, do_activation = True, pool_size = 1, postfix = 'conv5')
        self.conv6 = Conv(80, 80, filter_size = 3, stride = 1, do_activation = True, pool_size = 2, postfix = 'conv6')
        
        self.dense1 = Dense(64*80, 150, do_activation = True, postfix = 'dense1')
        self.dense2 = Dense(150, 10, do_activation = False, postfix = 'dense2')
        
    def logits(self, x):
        x_noise = tf.cond(self.use_noise, lambda: x + self.noise, lambda: x)
        conv_output = self.conv6(self.conv5(self.conv4(self.conv3(self.conv2(self.conv1(x_noise))))))
        flat = tf.reshape(conv_output, [-1, 64*80])
        return self.dense2(self.dense1(flat))
    
    @tf.function
    def __call__(self, x):
        return tf.nn.softmax(self.logits(x), axis = 1)

### 3.3: Compute cross-entropy loss
Write a `_loss` function to compute the cross-entropy loss.

In [21]:
def _loss(target, actual):
    loss = tf.nn.softmax_cross_entropy_with_logits(target, actual)
    return tf.reduce_mean(loss)

def _adv_loss(target, actual):
    return - _loss(target, actual)

def accuracy(target, actual):
    target_idx = tf.argmax(target, axis = 1)
    actual_idx = tf.argmax(actual, axis = 1)
    return tf.reduce_mean(tf.cast(tf.math.equal(target_idx, actual_idx),tf.float32))

### 3.4: Surrogate and adversarial loss and gradients
Add an optimizer and train method we'll use to train the surrogate model to act like the target model.
It should just minimize the cross-entropy loss between the model's predictions and the targets. When `use_noise` is false, we will not have any gradients to update the adversarial noise tensor with because it will not be referenced in our computation. This will cause TensorFlow to throw (very annoying) warning messages about nonexistent gradients. To suppress this, we can use this neat trick (credit [here](https://stackoverflow.com/questions/60022462/how-to-suppress-specific-warning-in-tensorflow-python)) to preprocess our gradients,
```
gradients = g.gradient(loss, model.trainable_variables)
surrogate_optimizer.apply_gradients( 
                            (grad, var) for (grad, var) in zip(gradients, model.trainable_variables)
                             if grad is not None)
```

When training the surrogate model, we want to change the model parameters to minimize the loss.
When creating the adversarial example, we want to do something very different: _change the adversarial noise tensor alone to maximize the loss_.

So, add an `adversarial=False` argument to your train method so we can toggle whether or not to train the surrogate model, or the adversarial noise.

Define a new adversarial loss which is the negative of the original loss, and a new optimizer to minimize it.
When you call `minimize()`, pass in the keyword argument `var_list=[model.noise]`, where "noise" is the name of your adversarial noise tensor.
This will prevent the optimizer from changing the weights of the model when we optimize the adversarial noise.

Add a summary scalar to plot the adversarial loss decreasing. Then, add a summary histogram and a summary image for the adversarial noise tensor, so we can plot it as we're learning the adversarial example.
Finally, add a summary image for the output of `cond`, which will be our adversarial example later.

In [24]:
optimizer = tf.optimizers.Adam()

def train(model, x, y, i, adversarial = False):
    if not adversarial:
        with tf.GradientTape() as g:
            loss = _loss(tf.squeeze(y), tf.squeeze(model.logits(x)))
        gradients = g.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients((grad, var) for (grad, var) in zip(gradients, model.trainable_variables) if grad is not None)
        with surr_writer.as_default():
            for gradient, variable in zip(gradients, model.trainable_variables):
                if variable.name and gradient is not None:
                    tf.summary.scalar('loss', loss, step = i)
                    tf.summary.histogram('gradients/' + variable.name, tf.norm(gradient), step=i)
                    tf.summary.histogram('variables/' + variable.name, tf.norm(variable), step=i)
    else:
        with tf.GradientTape() as g:
            adv_loss = _adv_loss(tf.squeeze(y), tf.sqeeze(model.logits(x)))
        gradients = g.gradient(adv_loss, [model.noise,])
        optimizer.apply_gradients((grad, var) for (grad, var) in zip(gradients, [model.noise,]) if grad is not None)
        with adv_writer.as_default():
            tf.summary.scalar('adversarial_loss', adv_loss, step = i)
            tf.summary.histogram('noise_tensorhistogram', tf.norm(model.noise), step = i)
            tf.summary.image('noise_tensor_image', tf.expand_dims(model.noise, axis = -1), step = i)
            tf.summary.image('cond_output', tf.expand_dims(tf.cond(model.use_noise, lambda: x + model.noise, lambda: x), axis = -1), step = i)
            
    acc = accuracy(y, model(x))
    return acc

## Section 4: Train the surrogate model
Train your surrogate model until it hits high accuracy.
At the very least it should have 90% accuracy -- I hit 98-100% on the training set.
Overfitting isn't really a concern here since we are actually trying to memorize the target model.

When the training loop is done, save the model under `./checkpoints/model_surrogate`.
We won't be modifying this particular checkpoint any more, it'll contain the fully-trained surrogate model with zero adversarial noise.

In [31]:
surrogate_model = Surrogate()
train_batch = 0
num_epochs = 10

for i in range(num_epochs):
    test_acc = []
    for x, y in dataset_train:
        if train_batch == 0:
            tf.summary.trace_on(graph=True, profiler=True)
            surrogate_model(x)
            with surr_writer.as_default():
                tf.summary.trace_export(name = 'first training batch', step = 0, profiler_outdir = 'dataset')
        acc = train(surrogate_model, x, y, train_batch)
        test_acc.append(acc)
        train_batch += 1
    print('Average Accuracy: ', np.mean(test_acc))
        
tf.train.Checkpoint(surrogate_model).save('./checkpoints/model_surrogate')

Average Accuracy:  0.44243777
Average Accuracy:  0.858428
Average Accuracy:  0.9341856
Average Accuracy:  0.9640151
Average Accuracy:  0.984375
Average Accuracy:  0.99289775
Average Accuracy:  0.99763256
Average Accuracy:  0.9957386
Average Accuracy:  0.99763256
Average Accuracy:  0.99857956


TypeError: __init__() takes 1 positional argument but 2 were given

## Section 5: Learn the adversarial example
Finally, it's time to create an adversarial example using our surrogate model.
Pick an image from the test set (or use the one I picked below), then run a training loop to minimize the adversarial loss (equivalently, maximize the model loss on that example).
To do this, use `surrogate_model.use_noise.assign(True)`.
This turns on the `use_noise` switch we set before, and overwrites the input image with the image we're going to turn into an adversarial example.

When it's done, save it to a new checkpoint, `./checkpoints/model_adversarial`.
This new checkpoint should have exactly the same model parameters as before (remember that we're only optimizing the noise), but a nonzero noise tensor.

At the end, also save the adversarial example (which will be contained in the result of your `cond`) and the noise tensor to a numpy array so we can use them later.

Periodically write the image and histogram summaries so you can look at them in TensorBoard later. 

You should be able to do this just by setting `adversarial=True` in your train method.

In [ ]:
# Pick and plot the image we'll turn into an adversarial example
idx = 1
img = x_test[idx]
lbl = y_test[idx]

plt.imshow(img[:, :, 0], cmap='gray')
plt.title('True label: {}'.format(np.argmax(lbl)))

In [ ]:
# Your code here

## Section 6: Visualize the adversarial example
Plot the original example, the adversarial example, and the adversarial noise below.

In [ ]:
# Your code here

## Section 7: Demonstrate that the surrogate model fails on the adversarial example
Compute the surrogate model's prediction and probability for the original example and the adversarial example, and compare them. Remember to set the `use_noise` switch to false when computing predictions.

In [ ]:
# Your code here

## 8: Demonstrate that the target model fails on the adversarial example
This is the real test -- transferring the learned adversarial example from the surrogate model to the target model.
Compute the target model's prediction and probability for the original example and the adversarial example, and compare them.

In [ ]:
# Your code here

# Congratulations, you've carried out a black-box adversarial attack!
These kinds of adversarial attacks are a serious concern in the real world, since they can be [made to work when printed on paper](https://blog.openai.com/robust-adversarial-inputs/) and [we're still bad at defending against them](https://blog.openai.com/adversarial-example-research/).

Some interesting things to try:
 - Did the surrogate model and the target model misclassify the adversarial example in the same way? (Mine did)
 - How subtle can you make the noise while still tricking the target network?
 - Look at how the adversarial example evolves through training in TensorBoard